In [3]:
### basic required packages ###
!pip install numpy
!pip install pyopenms
!pip install biopython
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 MB 5.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 4.8 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 1.8 MB/s eta 0:00:00a 0:00:01


In [ ]:
### GOOGLE AUTH ###
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip3 install --upgrade oauth2client 
!pip3 install --upgrade oauth2client 

In [197]:
#######################
###  Запускать тут ####
#######################

import glob
import pandas as pd
import os

# # CHANGE THIS
# EXPERIMENT_NAME = "KJE0006"

# Список для хранения обработанных DataFrame из каждого файла


def xlsx_paths(EXPERIMENT_NAME):
    # specifying the path to csv files
    path = f"./requests/{EXPERIMENT_NAME}"

    # csv files in the path
    file_paths = glob.glob(path + "/*.xlsx")
    return file_paths

def csv_paths(EXPERIMENT_NAME):
    # specifying the path to csv files
    path = f"./requests/{EXPERIMENT_NAME}"

    # csv files in the path
    file_paths = glob.glob(path + "/*.csv")
    return file_paths

def is_valid_sequence(seq):

    if pd.isna(seq):
        return False
    valid_chars = set("ATGCUYRSWKMBDHVN.-")
    return all(c in valid_chars for c in str(seq))

def process_excel_files(xlsx_paths, dfs_list):

    for file_path in xlsx_paths:
        # Чтение файла Excel
        df = pd.read_excel(file_path, header=None)

        if len(df.columns)==1:
            df = df[0].str.split(',', expand=True)

        if len(df.columns)<4:
            return print("ERROR IN INPUT FILE")

        # df curation
        for column in df.columns:
            df[column] = df[column].apply(lambda x: unicodedata.normalize('NFC', str(x)))
        #  pandas' Series.str.strip() method
        df = df.apply(lambda x: x.str.strip())
        df[0] = df[0].apply(lambda x: x.upper())

        # Получение имени самого левого столбца
        leftmost_column = df.columns[0]
        print(file_path, len(df))
        # Фильтрация строк согласно заданным условиям
        df = df[df[leftmost_column].apply(is_valid_sequence)]
        df = df.dropna()
        print(file_path, len(df))
        # Добавление обработанного DataFrame в список
        dfs_list.append(df)
        
        if dfs_list != None:
            pass
        else:
            dfs_list = []
            print([])
        
    return dfs_list
        
    
def process_csv_files(csv_paths, dfs_list):

    for file_path in csv_paths:
        # Чтение файла Excel
        df = pd.read_csv(file_path, header=None, sep=",")
        print(df)

        if len(df.columns)==1:
            df = df[0].str.split(',', expand=True)

        if len(df.columns)<4:
            return print("ERROR IN INPUT FILE")

        # df curation
        for column in df.columns:
            df[column] = df[column].apply(lambda x: unicodedata.normalize('NFC', str(x)))
        #  pandas' Series.str.strip() method
        df = df.apply(lambda x: x.str.strip())
        df[0] = df[0].apply(lambda x: x.upper())

        # Получение имени самого левого столбца
        leftmost_column = df.columns[0]
        print(file_path, len(df))
        # Фильтрация строк согласно заданным условиям
        df = df[df[leftmost_column].apply(is_valid_sequence)]
        df = df.dropna()
        # Добавление обработанного DataFrame в список
        dfs_list.append(df)
        
        if dfs_list != None:
            pass
        else:
            dfs_list = []
            print([])
            
    return dfs_list
        
def process_files(EXPERIMENT_NAME):
    dfs_list = []
    dfs_list = process_excel_files(xlsx_paths(EXPERIMENT_NAME), dfs_list)
    if dfs_list != None:
        print(dfs_list)
    else:
        dfs_list = []
    dfs_list = process_csv_files(csv_paths(EXPERIMENT_NAME), dfs_list)
    return dfs_list

# EXPERIMENT_NAME = input()
# dfs_list=process_files(EXPERIMENT_NAME)



In [198]:
def export_to_input_files(dfs_list, EXPERIMENT_NAME):
    # Конкатенация всех обработанных DataFrame по строкам
    final_df = None
    final_df = pd.concat(dfs_list, axis=0, join='outer', ignore_index=True)
    if (len(final_df)>384 and len(final_df)<768):

        df_A = final_df[:384]
        df_B = final_df[384:]

        # Сохранение результата в новый файл CSV
        df_A.to_csv(f"./result_input/{EXPERIMENT_NAME}_concatenated_file_plate_A.csv", index=False, sep = ',', header = None, encoding = "UTF-8")
        # Сохранение результата в новый файл Excel
        df_A.to_excel(f"./result_input/{EXPERIMENT_NAME}_concatenated_file_plate_A.xlsx", index=False, header = None, encoding = "UTF-8")

         # Сохранение результата в новый файл CSV
        df_B.to_csv(f"./result_input/{EXPERIMENT_NAME}_concatenated_file_plate_B.csv", index=False, sep = ',', header = None, encoding = "UTF-8")
        # Сохранение результата в новый файл Excel
        df_B.to_excel(f"./result_input/{EXPERIMENT_NAME}_concatenated_file_plate_B.xlsx", index=False, header = None, encoding = "UTF-8")

    else:

        # Сохранение результата в новый файл CSV
        final_df.to_csv(f"./result_input/{EXPERIMENT_NAME}_concatenated_file.csv", index=False, sep = ',', header = None, encoding = "UTF-8")

        # Сохранение результата в новый файл Excel
        final_df.to_excel(f"./result_input/{EXPERIMENT_NAME}_concatenated_file.xlsx", index=False, header = None, encoding = "UTF-8")

In [199]:
#!/usr/bin/env python
#
# File: DrOligoInputPreparation_withFormsAPI.py
# Author: Oleg Fedorov, Kamil Zaynullin
#
# Copyright (C) 2023 Group of automated synthesis
#
# This file is part of gene synthesis automatization pipeline implemented in SBM.

from __future__ import print_function

import io
import os

########################
## GOOGLE API IMPORTS ##
########################
from apiclient import discovery
from google.oauth2.credentials import Credentials
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from httplib2 import Http
from oauth2client import client, file, tools

SCOPES = "https://www.googleapis.com/auth/drive"
DISCOVERY_DOC = "https://forms.googleapis.com/$discovery/rest?version=v1"


def creating_dirs(EXPERIMENT_NAME):
    #################
    ## CREATE DIR  ##
    #################
    cwd = os.getcwd()
    dir = os.path.join(cwd, "requests", EXPERIMENT_NAME)
    if not os.path.exists(dir):
        os.mkdir(dir)


def google_auth():
    #################
    ## GOOGLE AUTH ##
    #################

    """
    caution! it appeares to be deprecated!!!

    """
    cwd = os.getcwd()
    token_file = os.path.join(cwd, "token.json")
    if os.path.exists(token_file):
        print("auth token already exists")
        store = file.Storage(token_file)
        creds = store.get()
    else:
        store = file.Storage("token.json")
        creds = None
        if not creds:
            flow = client.flow_from_clientsecrets("credentials.json", SCOPES)

            creds = tools.run_flow(flow, store)
    return creds


def retrive_forms_data(creds):
    ##################
    ## GOOGLE FORMS ##
    ##################
    service = discovery.build(
        "forms",
        "v1",
        http=creds.authorize(Http()),
        discoveryServiceUrl=DISCOVERY_DOC,
        static_discovery=False,
    )

    form_id = "1bVGyD-pz5w3BXM48yHWnhEDVnYNVgzRrF3ysvjAbqHQ"
    result = service.forms().responses().list(formId=form_id).execute()
    file_list = [
        response["answers"]["778e4a4f"]["fileUploadAnswers"]["answers"]
        for response in result["responses"]
    ]
    print(file_list)
    return file_list


####################################################
# downloading from Google Disk and writing to file #
####################################################
def download_request_file(fileID, filename, creds, EXPERIMENT_NAME):
    try:
        service = discovery.build("drive", "v3", credentials=creds)

        file_id = fileID

        # pylint: disable=maybe-no-member
        request = service.files().get_media(fileId=file_id)
        file = io.BytesIO()
        downloader = MediaIoBaseDownload(file, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}.")

    except HttpError as error:
        print(f"An error occurred: {error}")
        file = None
    filepath = os.path.join(os.getcwd(), "requests", EXPERIMENT_NAME, filename)
    with open(filepath, "wb") as outfile:
        # Copy the BytesIO stream to the output file
        outfile.write(file.getbuffer())

    return print(file.getvalue())


def download_all_files(creds, EXPERIMENT_NAME):
    for file in retrive_forms_data(creds):
        download_request_file(file[0]["fileId"], file[0]["fileName"], creds, EXPERIMENT_NAME)


##########
## main ##
##########
def main() -> None:
    EXPERIMENT_NAME = input("enter EXPERIMENT_NAME:")
    creating_dirs(EXPERIMENT_NAME)
    # retrive_forms_data()
    creds = google_auth()
    download_all_files(creds, EXPERIMENT_NAME)
    print("downloads finished")
    return EXPERIMENT_NAME

In [200]:
def combined_pipeline():
    EXPERIMENT_NAME = main()
    print(EXPERIMENT_NAME)
    dfs_list=process_files(EXPERIMENT_NAME)
    export_to_input_files(dfs_list, EXPERIMENT_NAME)
combined_pipeline()

enter EXPERIMENT_NAME:KJE0002
auth token already exists
[[{'fileId': '1N5ZrVtb6z-xT0V3hibB6i071vhDQMtnb', 'fileName': 'Праймеры для Mgal и pRS426 удлинённые - Глеб Фисунов.csv', 'mimeType': 'application/vnd.ms-excel'}]]
Download 100.
b'GATTCTACTGATAAAGCCG,DMT ON,dCas-F1,FISUNOV\r\nCAAAGCAATGAGATTCCC,DMT ON,dCas-R1,FISUNOV\r\nCAAGAGGTTAAAAGTGAGATCTGTG,DMT ON,MGA_WhiA-F,FISUNOV\r\nGCTGTTGTTCTTCTAGTTCGTG,DMT ON,MGA_WhiA-R,FISUNOV\r\nGCGATCTTAGCAGTATCAATGG,DMT ON,MGA_eno-F,FISUNOV\r\nACCTTCATCACCTTTGTTAGTA,DMT ON,MGA_eno-R,FISUNOV\r\nTTAAACAAGCCGCAATTAGGTC,DMT ON,MGA_RplC_F,FISUNOV\r\nTACCCTTCATAACTCTTTGAGC,DMT ON,MGA_RplC_R,FISUNOV\r\nATCGGATCCCGGGCCCGTCGACTGCAGAGGCGCGCTTGGCGTAATCATGGTC,DMT ON,pRS426G-F2,FISUNOV\r\nATCTAGATGCATTCGCGAGGTACCGAGCTCCAAGCTTATCGATACCGTCG,DMT ON,pRS426G-R2,FISUNOV\r\n\r\n\r\n'
downloads finished
KJE0002
./requests/KJE0002/ФОРМА ЗАКАЗА - Николай Постика.xlsx 4
./requests/KJE0002/ФОРМА ЗАКАЗА - Николай Постика.xlsx 1
./requests/KJE0002/Копия ФОРМА ЗАКАЗА 2 Калиш -